In [2]:
PATH_ = "C:\\Users\\ICN_admin\\Dropbox (Brain Modulation Lab)\\Shared Lab Folders\\CRCNS\\MOVEMENT DATA\\derivatives\\Int_old_grid\\"
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn import linear_model, metrics
from sklearn.model_selection import cross_val_score
import os 
from matplotlib import pyplot as plt
import pickle
import seaborn as sb
import pandas as pd
import numpy as np
from scipy import stats
import sys
sys.path.append('C:\\Users\\ICN_admin\\Documents\\icn\\icn_stats\\')
import icn_permutation_test

def append_time_dim(arr, y_, time_stamps):
    """
    apply added time dimension for the data array and label given time_stamps (with downsample_rate=100) in 100ms / need to check with 1375Hz
    """
    time_arr = np.zeros([arr.shape[0]-time_stamps, int(time_stamps*arr.shape[1])])
    for time_idx, time_ in enumerate(np.arange(time_stamps, arr.shape[0])):
        for time_point in range(time_stamps):
            time_arr[time_idx, time_point*arr.shape[1]:(time_point+1)*arr.shape[1]] = arr[time_-time_point,:]
    return time_arr, y_[time_stamps:]

def evalerror(preds, dtrain):
        """
        Custom defined r^2 evaluation function
        """
        labels = dtrain.get_label()
        # return a pair metric_name, result. The metric name must not contain a
        # colon (:) or a space since preds are margin(before logistic
        # transformation, cutoff at 0)

        r2 = metrics.r2_score(labels, preds)

        if r2 < 0:
            r2 = 0

        return 'r2', r2
    
def get_idx(l, contain_):
    L_ = []
    for idx, ind in enumerate(l):
        if len(contain_) > 1:
            for c in contain_:
                if c in ind:
                    L_.append(idx)
        else:
            if contain_[0] in ind:
                L_.append(idx)
    return L_

In [3]:
df_best = pd.read_pickle("LM_best_ECOGSTN_samesize.p")

In [5]:
df_best[:10]

,ch_,lat,loc,r2,sub
0,ECOG_RIGHT_0,CON,ECOG,0.540031,000
1,STN_RIGHT_1,CON,STN,0.530390,000
2,ECOG_RIGHT_1,IPS,ECOG,0.228433,000
3,STN_RIGHT_2,IPS,STN,0.068318,000
4,ECOG_RIGHT_19,CON,ECOG,0.340031,001
5,ECOG_LEFT_6,CON,ECOG,0.371472,001
6,STN_RIGHT_1,CON,STN,0.028266,001
7,STN_LEFT_0,CON,STN,0.111093,001
8,ECOG_RIGHT_24,IPS,ECOG,0.185061,001
9,ECOG_LEFT_10,IPS,ECOG,0.126492,001


In [ ]:
def get_best_idx():

In [ ]:
# COMBINATION ECOG
model = linear_model.LinearRegression()
df = pd.DataFrame()
time_stamps = 5

for ch_USED in [["ECOG"], ["STN"],["ECOG", "STN"]]:
    for f in range(len(os.listdir(PATH_))):
        sub_ = os.listdir(PATH_)[f][4:7]
        if sub_ not in ['000', '001', '004', '005', '006', '007', '008', '009', '010', '013', '014']:
            continue
        print(sub_)
        for hem in ["LEFT", "RIGHT"]:
            with open(PATH_+ '\\'+ os.listdir(PATH_)[f], 'rb') as handle:
                run_ = pickle.load(handle)
                print(run_["ch_names"])
                if len([ch for ch in run_["ch_names"] if "STN" in ch]) == 0:
                    continue
                print("PROCESS")
                X_ = []

                ch_idx = get_idx(run_["ch_names"], ch_USED)
                # the LABEL is not needed here... 
                # important step: mean of channels used! 
                X, y = append_time_dim(np.mean(run_["rf_data_median"][:,ch_idx,:], axis=1), \
                                       run_["label_baseline_corrected"][0,:], \
                                       time_stamps=time_stamps)
                #X__ = X.reshape(X.shape[1], X.shape[0]*X.shape[2], order="F")

                for label_idx, label in enumerate(run_["label_baseline_corrected"]):
                    res_ = np.mean(cross_val_score(model, X, label[time_stamps:], cv=3, scoring="r2", n_jobs=59))
                    if res_ < 0:
                        res_ = 0
                    if run_["label_con_true"][label_idx] is True:
                        lat = "CON"
                    else:
                        lat = "IPS"
                    if ch_USED == ["ECOG"]:
                        loc_here = "ECOG"
                    elif ch_USED == ["STN"]:
                        loc_here = "STN"
                    else:
                        loc_here = "ECOG_STN"
                    df = df.append({
                        "r2": res_,
                        "ch_": ch_,
                        "sub": sub_,
                        "loc": loc_here,
                        "lat": lat
                    }, ignore_index=True)
